In [390]:
import pandas as pd

In [391]:
j = 20

In [392]:
day_quotes = pd.read_csv(f'ES_ALL_QUOTES/es_{j}.csv')

In [393]:
# convert all futures prices to spot prices using a risk free rate of 2.7% (computed from sample data)
rate_factor = (15-j) / 365 if j < 15 else (107-j) / 365
day_quotes['Ask Price'] = day_quotes['Ask Price'] / (1+0.027*rate_factor)

In [394]:
day_quotes['Bid Price'] = day_quotes['Bid Price'] / (1+0.027*rate_factor)

In [395]:
day_quotes['Time'] = day_quotes['Time'].apply(lambda x: x + "000")
day_quotes.Time = pd.to_datetime(day_quotes.Time, format="%H:%M:%S.%f")

In [396]:
# shorten futures trading day to real trading day
day_quotes = day_quotes[(day_quotes['Time'] > '1900-01-01 09:30:00.000') & (day_quotes['Time'] < '1900-01-01 16:00:00.000')]

In [397]:
day_quotes

,Unnamed: 0,Symbol,Date,Time,Bid Price,Bid Size,Ask Price,Ask Size,Market Flag,Quote Condition
696967,20483824,ESZ13,09/20/2013,1900-01-01 09:30:00.006,1703.039888,360,1703.288290,557,E,NaN
696968,20483825,ESZ13,09/20/2013,1900-01-01 09:30:00.045,1703.039888,357,1703.288290,557,E,NaN
696969,20483826,ESZ13,09/20/2013,1900-01-01 09:30:00.054,1703.039888,358,1703.288290,557,E,NaN
696970,20483827,ESZ13,09/20/2013,1900-01-01 09:30:00.074,1703.039888,367,1703.288290,557,E,NaN
696971,20483828,ESZ13,09/20/2013,1900-01-01 09:30:00.074,1703.039888,368,1703.288290,557,E,NaN
...,...,...,...,...,...,...,...,...,...,...
1614938,21401795,ESZ13,09/20/2013,1900-01-01 15:59:54.750,1693.352235,42,1693.600636,154,E,NaN
1614939,21401796,ESZ13,09/20/2013,1900-01-01 15:59:54.750,1693.352235,42,1693.600636,152,E,NaN
1614940,21401797,ESZ13,09/20/2013,1900-01-01 15:59:56.083,1693.352235,41,1693.600636,152,E,NaN
1614941,21401798,ESZ13,09/20/2013,1900-01-01 15:59:57.471,1693.352235,41,1693.600636,153,E,NaN


In [398]:
column_names = [f"d{i+1}" for i in range(59)]
column_names.append("spot")


In [399]:
spx = pd.read_csv(f'SPX_1MIN_HIST/spx_{j}.csv')

In [400]:
spx = spx.drop(columns=['Unnamed: 0'])

In [401]:
# format: 2013-09-03 09:30:00
spx.datetime = pd.to_datetime(spx.datetime, format="%Y-%m-%d %H:%M:%S")

In [402]:
float(spx[spx.datetime == pd.Timestamp(2013, 9, j, 9, 30)][' close'])


1723.7

In [403]:
pd.Timestamp(2013, 9, 1) - pd.Timestamp(1900, 1, 1)

Timedelta('41516 days 00:00:00')

In [404]:
day_quotes['Time'] = day_quotes['Time'] + pd.Timedelta(41515+j, 'days')

In [405]:
day_quotes['Ask Price'][day_quotes.index[0]]

1703.2882898823736

In [406]:
condensed_1s = []
current_second_time = pd.Timestamp(2013, 9, j, 9, 30)
current_minute_time = pd.Timestamp(2013, 9, j, 9, 30)
start_px = (float(day_quotes['Ask Price'][day_quotes.index[0]]) + float(day_quotes['Bid Price'][day_quotes.index[0]])) / 2
for i, row in day_quotes.iterrows():
    # check if we are 59 seconds into the current minute
    if current_second_time + pd.Timedelta(1000, 'millis') == current_minute_time + pd.Timedelta(60000, 'millis'):
        # append spx price at current_minute_time + pd.Timedelta(60000, 'millis') to list
        # update minute and second
        current_minute_time = current_minute_time + pd.Timedelta(60000, 'millis')
        spx_price = float(spx[spx.datetime == current_minute_time][' close'])
        condensed_1s.append(spx_price - start_px)
        current_second_time = current_second_time + pd.Timedelta(1000, 'millis')
    elif row.Time > current_second_time + pd.Timedelta(1000, 'millis'):
        mid_px = (float(day_quotes['Ask Price'][i-1]) + float(day_quotes['Bid Price'][i-1])) / 2
        px_delta = mid_px - start_px
        condensed_1s.append(px_delta)
        current_second_time = current_second_time + pd.Timedelta(1000, 'millis')
        start_px = mid_px


In [407]:
len(condensed_1s)

23358

In [408]:
es_close_sequences = pd.DataFrame(columns=column_names, dtype=object)

In [409]:
i = 0
while i + 60 < len(condensed_1s): 
    es_close_sequences = es_close_sequences.append(pd.Series(condensed_1s[i:i+60], index=column_names), ignore_index=True)
    i+=60

In [410]:
len(condensed_1s)

23358

In [411]:
min(es_close_sequences.spot)

15.921966168411927

In [412]:
es_close_sequences.to_csv(f'cleaned_data/info_share/day{j}.csv')